<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Analisando-como-os-recursos-físicos-estão-distribuidos-nos-estados" data-toc-modified-id="Analisando-como-os-recursos-físicos-estão-distribuidos-nos-estados-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Analisando como os recursos físicos estão distribuidos nos estados</a></span></li><li><span><a href="#Analise-do-panorama-geral-dos-estados" data-toc-modified-id="Analise-do-panorama-geral-dos-estados-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Analise do panorama geral dos estados</a></span><ul class="toc-item"><li><ul class="toc-item"><li><span><a href="#Vamos-plotar-a-situação-dos-recursos-físicos-em-um-estado-específico" data-toc-modified-id="Vamos-plotar-a-situação-dos-recursos-físicos-em-um-estado-específico-2.0.1"><span class="toc-item-num">2.0.1&nbsp;&nbsp;</span>Vamos plotar a situação dos recursos físicos em um estado específico</a></span></li></ul></li><li><span><a href="#Escolhendo-um-novo-local-de-diagnostico" data-toc-modified-id="Escolhendo-um-novo-local-de-diagnostico-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Escolhendo um novo local de diagnostico</a></span><ul class="toc-item"><li><span><a href="#Agora-iremos-fazer-a-redistribuição-dos-recursos-físicos" data-toc-modified-id="Agora-iremos-fazer-a-redistribuição-dos-recursos-físicos-2.1.1"><span class="toc-item-num">2.1.1&nbsp;&nbsp;</span>Agora iremos fazer a redistribuição dos recursos físicos</a></span></li></ul></li></ul></li></ul></div>

# Analisando como os recursos físicos estão distribuidos nos estados

In [1]:
import numpy as np
import pandas as pd
from tqdm.auto import tqdm

import matplotlib.pyplot as plt
import mplleaflet

from geopy.distance import geodesic

import os

import math

from k_medoids_rf import KMedoidsWeighted

tqdm.pandas()

%matplotlib inline

In [2]:
data_dir = '../data/'
rf_dataset_name = 'RF- Mamógrafos.csv' #podemos fazer a analise tambem para tomografos
use_cols = ['COD_MUN','2018/Dez'] # obtendo os dados mais recentes
municipios_dataset_name = 'dados_mun_clean.csv'
distancia_max = 65 #padrão internacional que queremos atingir (para tomografos 70)
populacao_max = 240000 #padrão internacional que queremos atingir (para tomografos 100000)
UF_analisado = 'CE'

In [3]:
def dist_to_near_rf(row):
    '''
    Retorna a menor distancia a um recurso físico, dada a latitude e longitude de uma cidade
    '''
    distancias = cities_rf[cities_rf.UF == row.UF].apply(lambda x: geodesic((x['LATITUDE'], x['LONGITUDE']), (row.LATITUDE, row.LONGITUDE)).km , axis=1)
    index_min = distancias.idxmin()
    near_city = cities_rf.iloc[index_min]
    distancia = geodesic((near_city.LATITUDE, near_city.LONGITUDE), (row.LATITUDE, row.LONGITUDE)).km
    return near_city.COD_MUN, distancia

def population_around_rf(codigo_mun):
    '''
    Retorna a soma das populacoes das cidades ao redor de um rf
    '''
    cidades_proximas = municipios[municipios.COD_PROX == codigo_mun]
    return cidades_proximas.POPULACAO.astype(int).sum()

def generate_panorama_rfs(cities_rf, municipios):
    '''
    Cria um dataframe com um panorama geral dos estados
    '''
    total_cidades, total_populacao, total_rfs, ideal_rfs,suficiente,cidades_proximas, cidades_distantes, centros_suficiente, centros_insuficiente = [],[],[],[],[],[],[],[],[]
    for estado in estados:
        UF_analisado = estado
        cidades_no_estado = municipios[municipios.UF == estado]
        rf_no_estado = cities_rf[cities_rf.UF == estado]
        rf_no_estado = rf_no_estado.assign(POP_ATENDIDA = rf_no_estado.COD_MUN.apply(lambda cod: population_around_rf(cod)))

        distantes, proximas = cidades_no_estado[cidades_no_estado.DISTANCIAS > distancia_max], cidades_no_estado[cidades_no_estado.DISTANCIAS < distancia_max]
        suficientes, insuficientes = rf_no_estado[rf_no_estado.POP_ATENDIDA < populacao_max], rf_no_estado[rf_no_estado.POP_ATENDIDA > populacao_max]
        total_populacao_estado = cidades_no_estado.POPULACAO.astype(int).sum()
        total_rf_estado = rf_no_estado['2018/Dez'].astype(int).sum()
        ideal_rf_estado = math.ceil(total_populacao_estado/populacao_max)

        total_cidades.append(cidades_no_estado.COD_MUN.count())
        total_populacao.append(total_populacao_estado)
        total_rfs.append(total_rf_estado)
        ideal_rfs.append(ideal_rf_estado)
        suficiente.append(total_rf_estado > ideal_rf_estado)
        cidades_proximas.append(proximas.COD_MUN.count())
        cidades_distantes.append(distantes.COD_MUN.count())
        centros_suficiente.append(suficientes.COD_MUN.count())
        centros_insuficiente.append(insuficientes.COD_MUN.count())

    panorama_distribuicao_rfs_uf = pd.DataFrame({
        'UF': estados,
        'TOTAL_CIDADES': total_cidades,
        'TOTAL_POPULACAO': total_populacao,
        'TOTAL_MAMOGRAGOS': total_rfs,
        'IDEAL_RFS': ideal_rfs,
        'SUFICIENTE': suficiente,
        'CIDADES_PROXIMAS': cidades_proximas,
        'CIDADES_DISTANTES': cidades_distantes,
        'CENTROS_SUFICIENTES': centros_suficiente,
        'CENTROS_INSUFICIENTES': centros_insuficiente
    },columns= ['UF', 'TOTAL_MAMOGRAGOS', 'TOTAL_POPULACAO', 'IDEAL_RFS','SUFICIENTE','TOTAL_CIDADES', 'CIDADES_DISTANTES' , 'CIDADES_PROXIMAS', 'CENTROS_SUFICIENTES', 'CENTROS_INSUFICIENTES'])
    return panorama_distribuicao_rfs_uf


In [4]:
# Lendo os dataframes:

rf = pd.read_csv(os.path.join(data_dir, rf_dataset_name), encoding='latin', index_col=False, 
                     error_bad_lines=False, low_memory=False, usecols=use_cols)

municipios = pd.read_csv(os.path.join(data_dir, municipios_dataset_name), encoding='latin', index_col=0, 
                     error_bad_lines=False, low_memory=False)

municipios['COD_MUN'] = municipios['COD_MUN'].apply(lambda x: (int) ((x - x%10)/10))
cities_rf = pd.merge(rf[rf['2018/Dez'] > 0], municipios, on='COD_MUN', how='left')
municipios = municipios.assign(DISTANCIAS = municipios.progress_apply(lambda row: dist_to_near_rf(row)[1], axis=1), COD_PROX = municipios.progress_apply(lambda row: dist_to_near_rf(row)[0], axis=1))
estados = municipios.UF.unique()


# Analise do panorama geral dos estados

In [5]:
panorma_distribuicao_rfs_uf = generate_panorama_rfs(cities_rf, municipios)
panorma_distribuicao_rfs_uf

,UF,TOTAL_MAMOGRAGOS,TOTAL_POPULACAO,IDEAL_RFS,SUFICIENTE,TOTAL_CIDADES,CIDADES_DISTANTES,CIDADES_PROXIMAS,CENTROS_SUFICIENTES,CENTROS_INSUFICIENTES
0,RO,14,1768204,8,True,52,30,22,2,3
1,AC,3,803513,4,False,22,15,7,1,1
2,AM,55,3938336,17,True,62,16,46,35,1
3,RR,4,505665,3,True,15,12,3,0,1
4,PA,50,8206923,35,True,144,54,90,8,15
5,AP,2,766679,4,False,16,13,3,0,1
6,TO,18,1515126,7,True,139,83,56,5,2
7,MA,30,6904241,29,True,217,99,118,4,11
8,PI,40,3204028,14,True,224,91,133,9,4
9,CE,69,8904459,38,True,184,17,167,12,13


In [6]:
panorma_distribuicao_rfs_uf['rf_por_pop'] = populacao_max*(panorma_distribuicao_rfs_uf['TOTAL_MAMOGRAGOS']/panorma_distribuicao_rfs_uf['TOTAL_POPULACAO'])

In [7]:
panorma_distribuicao_rfs_uf

,UF,TOTAL_MAMOGRAGOS,TOTAL_POPULACAO,IDEAL_RFS,SUFICIENTE,TOTAL_CIDADES,CIDADES_DISTANTES,CIDADES_PROXIMAS,CENTROS_SUFICIENTES,CENTROS_INSUFICIENTES,rf_por_pop
0,RO,14,1768204,8,True,52,30,22,2,3,1.900233
1,AC,3,803513,4,False,22,15,7,1,1,0.896065
2,AM,55,3938336,17,True,62,16,46,35,1,3.351669
3,RR,4,505665,3,True,15,12,3,0,1,1.898490
4,PA,50,8206923,35,True,144,54,90,8,15,1.462180
5,AP,2,766679,4,False,16,13,3,0,1,0.626077
6,TO,18,1515126,7,True,139,83,56,5,2,2.851248
7,MA,30,6904241,29,True,217,99,118,4,11,1.042837
8,PI,40,3204028,14,True,224,91,133,9,4,2.996228
9,CE,69,8904459,38,True,184,17,167,12,13,1.859742


Da tabela acima, aplicando para mamografos, podemos notar que, com exceção de AC, AP e DF, todos os estados são suficientes de mamógrafos.

### Vamos plotar a situação dos recursos físicos em um estado específico

In [66]:
cidades_no_estado = municipios[municipios.UF == UF_analisado]
bool_dist = cidades_no_estado.DISTANCIAS < distancia_max
proximos, distantes = cidades_no_estado[bool_dist], cidades_no_estado[~bool_dist]

rf_no_estado = cities_rf[cities_rf.UF == UF_analisado]
rf_no_estado = rf_no_estado.assign(POP_ATENDIDA = rf_no_estado.COD_MUN.progress_apply(lambda cod: population_around_rf(cod)))
bool_pop = rf_no_estado.POP_ATENDIDA < populacao_max
pop_ok, pop_n_ok = rf_no_estado[bool_pop], rf_no_estado[~bool_pop]

In [68]:
fig, ax = plt.subplots(figsize=[10,10])
# plt.scatter(distantes.LONGITUDE, distantes.LATITUDE, color='r' , s = 70) # plotar cidades com mamografos distantes
# plt.scatter(proximos.LONGITUDE, proximos.LATITUDE, color='b',s = 70) # plotar cidades com mamografos proximos
plt.scatter(pop_ok.LONGITUDE, pop_ok.LATITUDE, marker='^', color='#5cd65c', s = 400) # plotar cidades em que o numero de mamografos sao adequados
plt.scatter(pop_n_ok.LONGITUDE, pop_n_ok.LATITUDE, marker='^', color='r', s = 400) #plotar cidades em que o numero de mamografos nao sao adequados
mplleaflet.display(fig=fig)

Em azul estão as cidades que conseguem ser atendidas pelos recursos físicos e em vermelho as que não conseguem.

Os triângulos verde claro são os recursos que estão sendo subutilizados enquanto que os verde escuro são os que estão sendo superutilizados

## Escolhendo um novo local de diagnostico

In [58]:
# aplicar o k-medoids para definir novos pontos
n_novos = 1 #numero de novos locais de diagnosticos

cidades_no_estado = municipios[municipios.UF == UF_analisado]
cidades_com_rf = cities_rf[cities_rf.UF == UF_analisado]['COD_MUN'].values.tolist()
X = cidades_no_estado[['LATITUDE', 'LONGITUDE', 'COD_MUN']].values.tolist()
medoids_fix_idxs = []
for i in range(len(X)):
    cod_mun = X[i][2]
    if cod_mun in cidades_com_rf:
        medoids_fix_idxs.append(i)

# quantidade inicial de cidades com rfs
n_centros = len(medoids_fix_idxs)

# quantidade de novas cidades com novo centro
clustering = KMedoidsWeighted(n_clusters=n_centros+n_novos)
medoids_idxs, clusters = clustering.fit(X, medoids_fix_idxs, max_itr=10, verbose=1)

Iteration: 0 ... loss: 51.31837760018
Iteration: 1 ... loss: 49.22550801167708


In [59]:
# adicionando cidades sugeridas na lista de cidades com rfs
novos_medoids = np.setdiff1d(medoids_idxs, medoids_fix_idxs)
for i in novos_medoids:
    cod_mun = int(X[i][2])
    nova_cidade = municipios[municipios.COD_MUN == cod_mun].values.tolist()[0]
    cities_rf = cities_rf.append({'COD_MUN' : cod_mun,
                                                '2018/Dez': 0,
                                                'UF': nova_cidade[0],
                                                'NOME': nova_cidade[2],
                                                'POPULACAO': nova_cidade[3],
                                                'LATITUDE': nova_cidade[4],
                                                'LONGITUDE': nova_cidade[5],
                                                'CAPITAL': nova_cidade[6],
                                                'CODIGO_UF': nova_cidade[7]
                                               } , ignore_index=True)



In [60]:
# atualizando distancias no municipio modificado
municipios = municipios.assign(DISTANCIAS = municipios.progress_apply(lambda row: (dist_to_near_rf(row)[1] if (row.UF == UF_analisado) else row.DISTANCIAS), axis=1), COD_PROX = municipios.progress_apply(lambda row: (dist_to_near_rf(row)[0] if (row.UF == UF_analisado) else row.COD_PROX), axis=1))
rf_no_estado = cities_rf[cities_rf.UF == UF_analisado]
rf_no_estado = rf_no_estado.assign(POP_ATENDIDA = rf_no_estado.COD_MUN.apply(lambda cod: population_around_rf(cod)))
rf_no_estado = rf_no_estado.assign(IDEAL_rf = rf_no_estado.POP_ATENDIDA.apply(lambda pop: math.ceil(pop/populacao_max)))

### Agora iremos fazer a redistribuição dos recursos físicos

In [61]:
rf_no_estado_copy = rf_no_estado.copy()

In [62]:
# preparando para a redistribuição os rfs

rf_no_estado_copy = rf_no_estado_copy.sort_values(by=['POP_ATENDIDA'], ascending=False)
total_rfs = rf_no_estado_copy['2018/Dez'].astype(int).sum()
total_cities = rf_no_estado_copy.COD_MUN.astype(int).count()
total_pop_atendida = rf_no_estado_copy.POP_ATENDIDA.astype(int).sum()
cities_matrix = rf_no_estado_copy[['COD_MUN', 'POP_ATENDIDA', 'IDEAL_rf']].values.tolist()
for i in range(total_cities):
    cities_matrix[i].append(0)

In [63]:
# redistribuir rfs se o estado for suficiente de rfs
if (total_pop_atendida / total_rfs) < populacao_max:
    aux = total_rfs
    i = 0
    saltos_seguidos = 0
    while (aux > 0) and (saltos_seguidos <= total_cities):
        if (cities_matrix[i][3] >= cities_matrix[i][2]):
            saltos_seguidos = saltos_seguidos + 1
            i = (i + 1)%total_cities
            continue
        saltos_seguidos = 0
        cities_matrix[i][3] = cities_matrix[i][3] + 1
        i = (i + 1)%total_cities
        aux = aux - 1
    i = 0
    while aux > 0:
        cities_matrix[i][3] = cities_matrix[i][3] + 1
        i = (i + 1)%total_cities
        aux = aux - 1 
    

In [64]:
cities_matrix = sorted(cities_matrix, key = lambda k: k[0])
redistirbuicao = []
for city in cities_matrix:
    redistirbuicao.append(city[-1])
    
rf_no_estado_copy = rf_no_estado_copy.sort_values(by='COD_MUN')
rf_no_estado_copy['REDISTRIBUICAO'] = redistirbuicao

In [65]:
#calculo as novas distancias aos recursos físicos
cidades_no_estado = cidades_no_estado.assign(NOVA_DISTANCIAS = cidades_no_estado.progress_apply(lambda row: dist_to_near_rf(row)[1], axis=1), COD_PROX = cidades_no_estado.progress_apply(lambda row: dist_to_near_rf(row)[0], axis=1))
proximos, distantes = cidades_no_estado[cidades_no_estado.NOVA_DISTANCIAS < distancia_max], cidades_no_estado[cidades_no_estado.NOVA_DISTANCIAS > distancia_max]
pop_ok, pop_n_ok = rf_no_estado_copy[rf_no_estado_copy.REDISTRIBUICAO >= rf_no_estado_copy.IDEAL_rf], rf_no_estado_copy[rf_no_estado_copy.REDISTRIBUICAO < rf_no_estado_copy.IDEAL_rf]

fig, ax = plt.subplots(figsize=[10,10])

# plt.scatter(distantes.LONGITUDE, distantes.LATITUDE, color='r' , s = 70) # plotar cidades com rfs distantes
# plt.scatter(proximos.LONGITUDE, proximos.LATITUDE, color='b',s = 70) # plotar cidades com rfs proximos
plt.scatter(pop_ok.LONGITUDE, pop_ok.LATITUDE, marker='^', color='#5cd65c', s = 400) # plotar cidades em que o numero de rfs sao adequados
plt.scatter(pop_n_ok.LONGITUDE, pop_n_ok.LATITUDE, marker='^', color='r', s = 400) #plotar cidades em que o numero de rfs nao sao adequados
mplleaflet.display(fig=fig)

/home/bruno/anaconda3/lib/python3.7/site-packages/IPython/core/display.py:694: UserWarning: Consider using IPython.display.IFrame instead
  warnings.warn("Consider using IPython.display.IFrame instead")


Como podemos ver no mapa acima houve uma redistirbuição dos recursos de forma a utilizar-los de forma mais otimizada.

In [27]:
#nova distribuição
rf_no_estado_copy

,COD_MUN,2018/Dez,UF,NOME,POPULACAO,LATITUDE,LONGITUDE,CAPITAL,CODIGO_UF,POP_ATENDIDA,IDEAL_rf,REDISTRIBUICAO
103,230110,1,CE,Aracati,72727,-4.55826,-37.7679,0,23,115752,1,1
104,230190,3,CE,Barbalha,58855,-7.29820,-39.3021,0,23,85927,1,1
945,230210,0,CE,BaturitÃ©,34949,-4.32598,-38.8812,0,23,350506,2,3
105,230250,2,CE,Brejo Santo,48056,-7.48469,-38.9799,0,23,187793,1,2
106,230260,1,CE,Camocim,62473,-2.90050,-40.8544,0,23,258837,2,3
107,230270,1,CE,Campos Sales,27123,-7.06761,-40.3687,0,23,161777,1,2
108,230350,1,CE,Cascavel,70047,-4.12967,-38.2412,0,23,161756,1,1
109,230370,2,CE,Caucaia,353932,-3.72797,-38.6619,0,23,353932,2,3
110,230410,3,CE,CrateÃºs,74271,-5.16768,-40.6536,0,23,260947,2,3
111,230420,3,CE,Crato,128680,-7.21530,-39.4103,0,23,187534,1,2


In [106]:
rf_no_estado_copy['DELTA'] = (rf_no_estado_copy['REDISTRIBUICAO'] - rf_no_estado_copy['2018/Dez'])